**<font color = black size=6>实验十二：神经网络</font>**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function
import os  

ModuleNotFoundError: No module named 'torch'

本实验使用Pytorch框架搭建神经网络，其他类似的框架还有TensorFlow。若同学对TensorFlow框架更为熟悉，可使用TensorFlow完成本次实验

**<font color = blue size=4>第一部分:PyTorch介绍</font>**

这里介绍一小部分PyTorch常用的库和函数，更多需求可参阅[PyTorch官方教程](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)以及[PyTorch官方文档](https://pytorch.org/docs/stable/index.html)。

In [ ]:
 print(torch.__version__) # 输出当前版本

**<font color = green size=3>1.Tensor</font>**

Tensor与NumPy中的ndarray很相似，但Tensor可以利用GPU来加速计算（本次实验中暂不涉及这部分内容）。

1.1. Tensor的创建

In [42]:
# 创建一个未初始化的Tensor
x = torch.empty(2, 3)
print(x)

# 从一个列表创建Tensor
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)

# 创建一个随机Tensor
x = torch.rand([3, 4])
print(x)

# 创建一个全零Tensor
x = torch.zeros([2, 3])
print(x)

# 创建一个全一Tensor
x = torch.ones([2, 3])
print(x)

tensor([[0.0000, 4.2969, 0.0000],
        [0.0000, 0.0000, 4.5977]])
tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[0.7687, 0.8341, 0.2797, 0.1409],
        [0.4442, 0.4342, 0.5008, 0.2835],
        [0.7328, 0.0959, 0.2500, 0.0992]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])


1.2. Tensor的运算

In [43]:
# 加减法
x = torch.tensor([[1,2,3],
                  [4,5,6]])
y = torch.tensor([[6,5,4],
                  [3,2,1]])
print(x + y)
print(x - y)

# 对应位置相乘
print(x * y)
print(x.mul(y))

# 矩阵乘法
print(x.matmul(y.T))
print(x @ y.T)

# reshape
print(x.reshape(3, 2))

# 拼接
print(torch.cat([x,y], dim=0)) # 纵向拼接
print(torch.cat([x,y], dim=1)) # 横向拼接

tensor([[7, 7, 7],
        [7, 7, 7]])
tensor([[-5, -3, -1],
        [ 1,  3,  5]])
tensor([[ 6, 10, 12],
        [12, 10,  6]])
tensor([[ 6, 10, 12],
        [12, 10,  6]])
tensor([[28, 10],
        [73, 28]])
tensor([[28, 10],
        [73, 28]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2, 3],
        [4, 5, 6],
        [6, 5, 4],
        [3, 2, 1]])
tensor([[1, 2, 3, 6, 5, 4],
        [4, 5, 6, 3, 2, 1]])


1.3. Tensor与ndarray的相互转换

In [44]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)

# 从Tensor转换到ndarray
y = x.numpy()
print(y)

# Tensor与ndarray是共享空间的
x[:]=0
print(y)

# 从ndarray到Tensor
z = torch.from_numpy(y)
print(z)

tensor([[1, 2, 3],
        [4, 5, 6]])
[[1 2 3]
 [4 5 6]]
[[0 0 0]
 [0 0 0]]
tensor([[0, 0, 0],
        [0, 0, 0]])


**<font color = green size=3>2.梯度计算</font>**

2.1 梯度计算

In [45]:
#定义变量
a = torch.tensor([[1., 2.]], requires_grad=True)
b = torch.tensor([[3.], [4.]])
c = torch.tensor(5., requires_grad=True)

#计算输出
z = a @ b + c

#自动计算梯度
z.backward()

#输出叶子节点的梯度
print(a.grad) #z对a的梯度
print(b.grad) #由于b默认requires_grad为false，因此无法计算梯度，输出为None
print(c.grad) #z对c的梯度

tensor([[3., 4.]])
None
tensor(1.)


2.2 梯度清零

In [46]:
#支持多种运算求梯度，如torch.mean(),torch.sum()等
a = torch.ones(20, requires_grad=True)
z = torch.sum(torch.sigmoid(a))
z.backward()
print("梯度（a.grad）:", a.grad)


#多次求梯度时梯度会累加，可使用tensor.grad.zero_()进行手动清零
x = torch.tensor(2., requires_grad=True)
y = x ** 2
y.backward()
print("求梯度后的结果（x.grad）:", x.grad)

z = x + 3
x.grad.zero_()  #可以将这句进行手动清零的代码注释掉后查看输出结果，来看到梯度清零的作用
z.backward()
print("求梯度后的结果（x.grad）:", x.grad)


梯度（a.grad）: tensor([0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966,
        0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966, 0.1966,
        0.1966, 0.1966])
求梯度后的结果（x.grad）: tensor(4.)
求梯度后的结果（x.grad）: tensor(1.)


**<font color = green size=3>3. 神经网络</font>**

3.1 神经网络的构建

In [58]:
# 定义神经网络模型，继承自nn.Module
class Net(nn.Module):
    #输入层的维度为 input_dim
    #隐藏层的维度为 hidden_dim
    #输出层的维度为 output_dim
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        #激活函数relu，用于在全连接层之间加入非线性变换
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out1 = self.relu(out)
        out2 = self.fc2(out1)
        return out1,out2


# 创建神经网络模型实例并输出
net = Net(10,5,1)
print(net)

Net(
  (fc1): Linear(in_features=10, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=1, bias=True)
  (relu): ReLU()
)


3.2 神经网络参数查询

In [48]:
# 该神经网络中可学习的参数可以通过net.parameters()访问
params = list(net.parameters())
print([params[i].size() for i in range(len(params))])  
print("Parameters:",params)

[torch.Size([5, 10]), torch.Size([5]), torch.Size([1, 5]), torch.Size([1])]
Parameters: [Parameter containing:
tensor([[-1.0985e-01, -7.6312e-02, -2.0128e-01, -1.1968e-02, -2.6385e-01,
          2.5134e-01,  1.8929e-01,  1.4129e-01,  5.4100e-02,  2.4919e-01],
        [-1.6939e-01,  3.0797e-03,  1.2730e-01, -2.8699e-01,  6.4637e-02,
          1.6762e-01, -3.6328e-02,  3.0643e-01, -2.8438e-01, -2.4651e-01],
        [-7.0283e-02,  3.1723e-02, -8.8639e-02, -2.5455e-01,  2.6949e-01,
         -1.3448e-01,  1.1654e-01,  2.2735e-01, -2.1453e-01,  2.8511e-01],
        [ 4.0381e-02, -2.4621e-01, -3.1451e-01,  1.6439e-01,  2.6017e-01,
          1.4010e-01,  7.0870e-05,  2.5389e-01, -1.5460e-01,  8.2921e-02],
        [ 1.4023e-01,  1.8721e-01, -7.6787e-02, -3.3322e-02, -2.2830e-01,
         -1.8822e-01, -2.1006e-01,  1.5386e-01, -2.8845e-01, -2.5285e-01]],
       requires_grad=True), Parameter containing:
tensor([ 0.0196,  0.1281, -0.0757,  0.1749, -0.0495], requires_grad=True), Parameter containi

3.3 神经网络前向传播

In [59]:
net.eval()
#输入维度为10，生成数据
input=torch.ones([1,10])
input=input.float()

# 进行一次forward()前向传播
output1, output2  = net(input) 

# 前向传播并输出每一层的输出值
print("Output of first layer:", output1)
print("Output of second layer:", output2)

Output of first layer: tensor([[0.4005, 0.0000, 0.2670, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
Output of second layer: tensor([[-0.2747]], grad_fn=<AddmmBackward0>)


3.4 神经网络反向传播

In [60]:
loss_fn = nn.MSELoss()
target = torch.randn(1, 1)
loss = loss_fn(output2, target)

# 反向传播并输出每一层的梯度
net.zero_grad()
loss.backward()

print("Gradients of first layer:")
print(net.fc1.weight.grad)
print(net.fc1.bias.grad)

print("Gradients of second layer:")
print(net.fc2.weight.grad)
print(net.fc2.bias.grad)

Gradients of first layer:
tensor([[-0.1460, -0.1460, -0.1460, -0.1460, -0.1460, -0.1460, -0.1460, -0.1460,
         -0.1460, -0.1460],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.1637,  0.1637,  0.1637,  0.1637,  0.1637,  0.1637,  0.1637,  0.1637,
          0.1637,  0.1637],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000]])
tensor([-0.1460,  0.0000,  0.1637,  0.0000,  0.0000])
Gradients of second layer:
tensor([[0.3021, 0.0000, 0.2014, 0.0000, 0.0000]])
tensor([0.7542])


3.5 训练神经网络的全过程例子

In [61]:
# 创建一个简单的线性回归模型
model = nn.Linear(1, 1)
print(list(model.parameters()))
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

Epoch=10

#生成数据
inputs = torch.tensor([[1.0], [2.0], [3.0]])
labels = torch.tensor([[2.0], [4.0], [6.0]])

# 模拟训练过程
for epoch in range(Epoch):
    # 模拟输入数据和标签
    
    # 前向传播
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)
    
    #梯度清零
    optimizer.zero_grad()
    
    
    # 反向传播
    loss.backward()

    
    # 更新参数
    optimizer.step()

    # 打印梯度值
    print('Epoch [{}/{}], Loss: {}'.format(epoch+1,Epoch, loss),'. Gradient: {}'.format(model.weight.grad))
    #print('Gradient: {}'.format(model.weight.grad))


[Parameter containing:
tensor([[0.3990]], requires_grad=True), Parameter containing:
tensor([0.0202], requires_grad=True)]
Epoch [1/10], Loss: 11.833146095275879 . Gradient: tensor([[-14.8622]])
Epoch [2/10], Loss: 9.364291191101074 . Gradient: tensor([[-13.2205]])
Epoch [3/10], Loss: 7.412741184234619 . Gradient: tensor([[-11.7609]])
Epoch [4/10], Loss: 5.870094299316406 . Gradient: tensor([[-10.4632]])
Epoch [5/10], Loss: 4.650662422180176 . Gradient: tensor([[-9.3094]])
Epoch [6/10], Loss: 3.686716079711914 . Gradient: tensor([[-8.2837]])
Epoch [7/10], Loss: 2.9247169494628906 . Gradient: tensor([[-7.3716]])
Epoch [8/10], Loss: 2.322347402572632 . Gradient: tensor([[-6.5608]])
Epoch [9/10], Loss: 1.8461552858352661 . Gradient: tensor([[-5.8398]])
Epoch [10/10], Loss: 1.4697006940841675 . Gradient: tensor([[-5.1989]])


3.5 神经网络参数更新

1) 用梯度下降法(手动)更新net中的参数

In [62]:
#for f in net.parameters():
    #f.data.sub_(f.grad.data * learning_rate)

2) 用PyTorch的优化器来更新net中的参数

In [63]:
#选择优化器
#optimizer = optim.SGD(net.parameters(), lr=0.01)

# 建立循环:
#optimizer.zero_grad()             # 梯度清零
#output = net(input)               # 前向传播
#loss = criterion(output, target)  # 计算误差
#loss.backward()                   # 后向传播
#optimizer.step()                  # 参数更新

**<font color = blue size=4>第二部分:实验内容</font>**

[Red Wine Quality](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009)是一个关于红酒品质的数据集，总共有1599个样本，每个样本包含11个(都是连续的)特征以及1个标签，每个标签的取值是连续的。本次实验已经按照8：2的比例划分成了训练数据集'wine_train.csv'以及测试数据集'wine_test.csv'，且每个数据集都已经做了归一化处理。

<span style="color:purple">1) 读入训练数据集'wine_train.csv'与测试数据集'wine_test.csv'。</span>

In [64]:
#your code here
wine_train = pd.read_csv("wine_train.csv")
wine_test = pd.read_csv("wine_test.csv")

(1279, 12)
(320, 12)


<span style="color:purple">2) 利用线性层和激活函数搭建一个神经网络，要求输入和输出维度与数据集维度一致，而神经网络深度、隐藏层大小、激活函数种类等超参数自行调整。</span>

In [ ]:
input_dim = wine_train.shape[1] - 1
hidden_dim = 5
output_dim = 1
# print(input_dim)


# 定义神经网络模型，继承自nn.Module
class Net(nn.Module):
    # 输入层的维度为 input_dim
    # 隐藏层的维度为 hidden_dim
    # 输出层的维度为 output_dim
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # 激活函数relu，用于在全连接层之间加入非线性变换
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out1 = self.relu(out)
        out2 = self.fc2(out1)
        return out1, out2

<span style="color:purple">3) 用PyTorch的优化器(随机梯度下降)来进行模型参数更新，记下每轮迭代中的训练损失和测试损失。</span>

In [2]:
#your code here

<span style="color:purple">4) 画出训练损失和测试损失关于迭代轮数的折线图。</span>

In [3]:
#your code here

**<font color = blue size=4>第三部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可

二、本次实验分为两周完成，实验报告提交截止日期: 12月15号14:20  
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  

实验十二(神经网络)的实验报告上交地址: https://send2me.cn/Wk9FsyYO/SKCBsWeFtvwQsg

三、课堂课件获取地址:https://www.jianguoyun.com/p/DWHYtsEQp5WhChjwtKoFIAA
实验内容获取地址:https://www.jianguoyun.com/p/DbNY_SIQp5WhChjvtKoFIAA